In [2]:
%cd /Users/maayanlab/Documents/projects/human-melanoma-signature
import sys
sys.path.append('pipeline/scripts/')
import pandas as pd
import h5py
import numpy as np
import json, operator, math, requests
from geode_jupies import *
import sklearn.metrics.pairwise as smp
from scipy.spatial.distance import cosine
import seaborn as sns; sns.set(color_codes=True)
import RNAseq as RNA
from collections import Counter
import time

/Users/maayanlab/Documents/projects/human-melanoma-signature


/Users/maayanlab/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
%pwd

'/Users/maayanlab/Documents/projects/human-melanoma-signature'

In [4]:
from Melanoma import *

# Code Development for Melanoma Pipeline
## Define input and output files

In [51]:
# infiles S4.4
#infile = 's4-enrichment.dir/melanoma-enrichr-results.txt'
#infile2 = 's4-enrichment.dir/enrichr-results-top.txt'

In [22]:
# infiles
# infile1 = 's2-signatures.dir/melanoma-intersection-2000.txt'
# infile2 = 's2-signatures.dir/melanoma-intersection-3000.txt'
# infile3 = 's2-signatures.dir/melanoma-intersection-5000.txt'

In [33]:
# infiles
infile='s2-signatures.dir/melanoma-geneset-top.json'

In [128]:
# infiles S2.2
sig = 's2-signatures.dir/melanoma-signatures.txt'
outfile = 's2-signatures.dir/melanoma-signatures-0.6+-.txt'

In [5]:
# #infiles
# cpm = 's1-expression_data.dir/melanoma-cpm.txt'
# matches = 's2-expression_data.dir/melanoma-idmatches.txt'

In [7]:
# # Infile & outfile
# infiles = ['s1-expression_data.dir/melanoma-cpm.txt']
# outfile = 's1-expression_data.dir/melanoma-IDstatus.txt'

In [8]:
# # Infile & outfile
# infiles = ['rawdata/human_matrix.h5', 'rawdata/m2.csv']
# outfile = 's1-expression_data.dir/melanoma-cpm.txt'

### Test Code

In [6]:
sig_df=pd.read_table(sig, index_col='gene')

In [ ]:
# export= set(sorted_genes_1) | set(sorted_genes_2) | set(sorted_genes_3) | set(sorted_genes_4) | set(sorted_genes_5) | set(sorted_genes_6)
# test1=sig_df.loc[export]
center= set(sorted_genes_1) & set(sorted_genes_2) & set(sorted_genes_3) & set(sorted_genes_4) & set(sorted_genes_5) & set(sorted_genes_6)
top500inter=sig_df.loc[center]
top500inter.to_csv(outfile, sep='\t')

In [34]:
with open(infile) as openfile:
    dict_top_genes = json.load(openfile)

In [130]:
sig_df_ranked=sig_df.rank()

In [95]:
like=np.zeros_like(sig_df_ranked)

In [114]:
like[sig_df_ranked<250]=1
like[sig_df_ranked>(18992-250)]= -1

In [115]:
ls=like.sum(axis=1)#row sums of all 18992 row entries

In [127]:
subset80=sig_df[ls>6].append(sig_df[ls<-6])
subset80

,1,11,15_a,15_b,15_c,17_a,17_b,17_c,18_a,18_b,...,19_b,19_c,19_d,2_a,2_b,2_c,3,5_a,5_b,5_c
gene,,,,,,,,,,,,,,,,,,,,,
APOE,-0.000016,-0.000613,-0.000064,0.000063,-0.000195,-0.013925,-0.013557,-0.014242,0.000166,-0.000522,...,0.000112,-0.000078,0.000129,-0.031531,-0.005672,-0.008321,0.001155,0.000588,-0.019798,-0.011031
BCAN,0.000056,-0.000481,-0.000686,-0.000325,0.000146,-0.022273,-0.018527,-0.019219,0.000127,-0.000602,...,0.000162,-0.000062,0.000130,-0.015221,-0.002337,-0.003172,-0.001047,0.000940,-0.057721,-0.016005
CD63,-0.003033,0.009523,0.013134,-0.003645,-0.005263,0.020560,-0.034753,-0.002619,-0.004782,0.003835,...,-0.107659,-0.052226,-0.142066,-0.051926,-0.024105,-0.040364,0.005264,-0.047726,-0.049897,-0.021544
CENPF,-0.015469,0.016180,0.002824,0.004797,-0.000977,0.010666,0.001214,0.006883,-0.006136,-0.004696,...,-0.015093,-0.011386,-0.005435,-0.010460,-0.003949,-0.003850,-0.010479,-0.026524,0.001118,-0.013007
CTSD,0.009278,-0.001009,-0.014479,-0.053563,-0.027008,0.008925,-0.013649,-0.016122,-0.000302,0.001727,...,-0.000011,-0.000254,0.000881,-0.026845,-0.028360,-0.026912,0.019353,-0.021392,-0.065970,-0.000407
DCT,0.000052,-0.000738,-0.000799,-0.000352,0.000102,-0.075731,-0.124897,-0.115012,0.000189,-0.003240,...,0.000162,-0.000056,0.000130,-0.109246,-0.002155,-0.002932,-0.000548,-0.004324,-0.169040,-0.574152
EDNRB,-0.000104,-0.000279,0.005696,0.004385,-0.009191,-0.010521,-0.014488,-0.012309,0.000423,-0.009286,...,0.000143,-0.000078,0.000175,-0.006746,-0.002516,-0.003039,0.004365,0.001536,-0.042531,-0.010944
FDFT1,0.001202,0.003418,-0.001562,-0.003281,-0.011731,-0.003668,0.000140,-0.000946,0.000488,0.001078,...,-0.011070,-0.009550,-0.003641,-0.006234,0.001937,0.000728,-0.000105,-0.004272,-0.038538,-0.007698
FSTL1,-0.006887,0.001544,-0.005480,0.002262,-0.006611,-0.015046,-0.018402,-0.014688,0.000258,0.007551,...,0.003313,0.059977,0.013189,-0.030691,-0.006845,-0.009990,0.013758,0.020037,-0.019755,-0.011249


In [129]:
subset80.to_csv(outfile, sep='\t')

In [120]:
sig_df[ls<-6]

,1,11,15_a,15_b,15_c,17_a,17_b,17_c,18_a,18_b,...,19_b,19_c,19_d,2_a,2_b,2_c,3,5_a,5_b,5_c
gene,,,,,,,,,,,,,,,,,,,,,
ARF1,0.013464,0.011189,0.011114,-0.005232,0.009224,0.003709,0.002844,0.007950,0.001609,0.012951,...,-0.000895,-0.000173,-0.001236,0.006337,0.017768,0.014118,0.017558,-0.007151,0.001008,0.001341
ATP5A1,0.005827,0.001717,0.012249,0.008913,-0.002982,0.002474,0.004607,0.004305,-0.002445,0.005607,...,0.014225,0.008314,-0.000867,-0.002609,0.012874,0.019582,-0.001087,-0.011512,-0.008384,0.012971
ATP5B,0.017094,0.015094,0.012006,0.003951,0.003839,0.028793,0.016247,0.044232,-0.006942,-0.004925,...,0.000921,0.002301,-0.002076,-0.016051,0.023867,0.027339,0.029712,0.002077,-0.014387,0.024539
CALU,-0.014243,0.012260,0.015804,0.020069,-0.022787,0.048300,0.000591,0.038657,0.003102,0.086831,...,0.001369,0.007032,-0.007285,-0.005448,0.014381,0.019440,0.017307,-0.012601,-0.035847,-0.006634
CDKN1A,0.017719,0.011552,-0.006558,-0.005600,0.004963,-0.002938,0.008185,0.012501,0.001304,0.024517,...,0.005835,0.003552,0.002882,0.000246,0.007296,0.038823,0.006773,0.077217,0.033937,0.025822
EIF4G2,-0.005112,0.005748,0.009975,0.016972,0.016569,0.041197,0.040805,0.018211,-0.001576,0.019138,...,0.003269,0.006447,-0.003939,0.014385,0.012329,0.039707,0.000924,-0.015259,-0.000162,0.007326
FKBP10,-0.004427,0.008843,0.005591,0.001214,-0.001030,0.006460,0.003839,0.019351,-0.001683,0.014262,...,0.000105,0.000073,0.000041,-0.005699,-0.002614,0.015101,0.027380,0.004518,0.021991,0.009799
MDM2,0.000301,0.010822,-0.002552,0.000257,0.003334,0.001115,0.001935,0.006729,0.001876,0.002434,...,0.023692,0.146962,-0.056711,0.000587,0.014778,0.018533,-0.003985,0.007102,0.002293,0.014859
NCL,-0.020743,0.006199,0.013428,0.016320,0.003180,0.050507,0.032309,0.033484,-0.021761,0.025569,...,0.012944,0.007609,0.002933,-0.024967,0.010323,0.004723,0.000185,-0.006624,0.007019,0.007834


In [126]:
sig_df[ls>6] 

,1,11,15_a,15_b,15_c,17_a,17_b,17_c,18_a,18_b,...,19_b,19_c,19_d,2_a,2_b,2_c,3,5_a,5_b,5_c
gene,,,,,,,,,,,,,,,,,,,,,
APOE,-0.000016,-0.000613,-0.000064,0.000063,-0.000195,-0.013925,-0.013557,-0.014242,0.000166,-0.000522,...,0.000112,-0.000078,0.000129,-0.031531,-0.005672,-0.008321,0.001155,0.000588,-0.019798,-0.011031
BCAN,0.000056,-0.000481,-0.000686,-0.000325,0.000146,-0.022273,-0.018527,-0.019219,0.000127,-0.000602,...,0.000162,-0.000062,0.000130,-0.015221,-0.002337,-0.003172,-0.001047,0.000940,-0.057721,-0.016005
CD63,-0.003033,0.009523,0.013134,-0.003645,-0.005263,0.020560,-0.034753,-0.002619,-0.004782,0.003835,...,-0.107659,-0.052226,-0.142066,-0.051926,-0.024105,-0.040364,0.005264,-0.047726,-0.049897,-0.021544
CENPF,-0.015469,0.016180,0.002824,0.004797,-0.000977,0.010666,0.001214,0.006883,-0.006136,-0.004696,...,-0.015093,-0.011386,-0.005435,-0.010460,-0.003949,-0.003850,-0.010479,-0.026524,0.001118,-0.013007
CTSD,0.009278,-0.001009,-0.014479,-0.053563,-0.027008,0.008925,-0.013649,-0.016122,-0.000302,0.001727,...,-0.000011,-0.000254,0.000881,-0.026845,-0.028360,-0.026912,0.019353,-0.021392,-0.065970,-0.000407
DCT,0.000052,-0.000738,-0.000799,-0.000352,0.000102,-0.075731,-0.124897,-0.115012,0.000189,-0.003240,...,0.000162,-0.000056,0.000130,-0.109246,-0.002155,-0.002932,-0.000548,-0.004324,-0.169040,-0.574152
EDNRB,-0.000104,-0.000279,0.005696,0.004385,-0.009191,-0.010521,-0.014488,-0.012309,0.000423,-0.009286,...,0.000143,-0.000078,0.000175,-0.006746,-0.002516,-0.003039,0.004365,0.001536,-0.042531,-0.010944
FDFT1,0.001202,0.003418,-0.001562,-0.003281,-0.011731,-0.003668,0.000140,-0.000946,0.000488,0.001078,...,-0.011070,-0.009550,-0.003641,-0.006234,0.001937,0.000728,-0.000105,-0.004272,-0.038538,-0.007698
FSTL1,-0.006887,0.001544,-0.005480,0.002262,-0.006611,-0.015046,-0.018402,-0.014688,0.000258,0.007551,...,0.003313,0.059977,0.013189,-0.030691,-0.006845,-0.009990,0.013758,0.020037,-0.019755,-0.011249


In [64]:
sig_df_ranked.nlargest(250,columns='1')

,1,11,15_a,15_b,15_c,17_a,17_b,17_c,18_a,18_b,...,19_b,19_c,19_d,2_a,2_b,2_c,3,5_a,5_b,5_c
gene,,,,,,,,,,,,,,,,,,,,,
COL1A1,18992.0,9798.0,9897.0,11339.0,5949.0,143.0,307.0,211.0,12877.0,13684.0,...,120.0,18991.0,1.0,15343.0,7055.0,6904.0,16203.0,527.0,33.0,709.0
COL5A1,18991.0,12867.0,242.0,508.0,18196.0,1453.0,18121.0,9523.0,1404.0,16720.0,...,286.0,18881.0,114.0,17447.0,5543.0,4848.0,18838.0,15952.0,1779.0,4021.0
COL6A1,18990.0,915.0,1039.0,1715.0,14113.0,60.0,133.0,130.0,264.0,18760.0,...,1699.0,1376.0,15369.0,3124.0,17362.0,15777.0,18887.0,18481.0,18713.0,18609.0
FLNA,18989.0,18979.0,81.0,45.0,18978.0,18708.0,18987.0,1594.0,6.0,18974.0,...,13110.0,5785.0,16148.0,18976.0,18849.0,18983.0,18970.0,16.0,49.0,8.0
MMP2,18988.0,16717.0,488.0,834.0,16384.0,26.0,16995.0,49.0,253.0,18096.0,...,16591.0,15803.0,18335.0,1892.0,13173.0,14091.0,18927.0,208.0,42.0,38.0
ACTG1,18987.0,18663.0,18273.0,10.0,2637.0,18981.0,18956.0,95.0,7.0,18905.0,...,18990.0,18985.0,8.0,75.0,18975.0,18978.0,18982.0,5.0,14.0,5.0
MT-ND4,18986.0,13846.0,18979.0,18985.0,9.0,12.0,7.0,17.0,108.0,35.0,...,18913.0,18807.0,18787.0,18985.0,18984.0,18946.0,62.0,18981.0,16.0,14.0
LGALS1,18985.0,18931.0,156.0,59.0,279.0,18934.0,18742.0,18914.0,1027.0,12028.0,...,884.0,601.0,160.0,1600.0,18873.0,18864.0,628.0,33.0,131.0,41.0
COL6A2,18984.0,14078.0,8841.0,1926.0,13913.0,62.0,138.0,132.0,505.0,1073.0,...,4472.0,3352.0,13168.0,4411.0,13006.0,12154.0,18913.0,18219.0,18734.0,4512.0


In [ ]:
TF=sig_df.isin(test1)

In [65]:
from IPython.display import HTML, display
# to display hyperlink as <a> tag in output cells
def display_link(url):
    raw_html = '<a href="%s" target="_blank">%s</a>' % (url, url)
    return display(HTML(raw_html))

In [66]:
clustergrammer_url = 'http://amp.pharm.mssm.edu/clustergrammer/matrix_upload/'
r = requests.post(clustergrammer_url, files={'file': open(outfile, 'rb')})
link = r.text
display_link(link)

In [67]:
from IPython.display import IFrame
display(IFrame(link, width="1000", height="1000"))

In [234]:
c = Counter()
for i in [enr_res[signature]['top'] for signature in enr_res.keys()]:
    c.update(i['term_name'].iloc[:5])

In [235]:
c.most_common()

[('protein targeting to ER (GO:0045047)', 17),
 ('SRP-dependent cotranslational protein targeting to membrane (GO:0006614)',
  17),
 ('nuclear-transcribed mRNA catabolic process, nonsense-mediated decay (GO:0000184)',
  17),
 ('viral transcription (GO:0019083)', 13),
 ('peptide biosynthetic process (GO:0043043)', 8),
 ('viral process (GO:0016032)', 5),
 ('neutrophil degranulation (GO:0043312)', 4),
 ('extracellular matrix organization (GO:0030198)', 3),
 ('neutrophil mediated immunity (GO:0002446)', 3),
 ('regulated exocytosis (GO:0045055)', 2),
 ('platelet degranulation (GO:0002576)', 2),
 ('rRNA metabolic process (GO:0016072)', 2),
 ('mitochondrial ATP synthesis coupled proton transport (GO:0042776)', 1),
 ('ribosomal small subunit export from nucleus (GO:0000056)', 1),
 ('positive regulation of nitric oxide biosynthetic process (GO:0045429)', 1),
 ('positive regulation of nitric oxide metabolic process (GO:1904407)', 1),
 ('ER to Golgi vesicle-mediated transport (GO:0006888)', 1),
 

In [ ]:
c = Counter()
for i in [results[signature]['top'] for signature in results.keys()]
for i in [results['1']['top']]: 
    c.update(i['Term name'].iloc[:5])
[results[signature]['top'] for signature in results.keys()]

In [176]:
# Create a 3rd dictionary 
from collections import defaultdict
results = defaultdict(dict)
# Iterate through dictionary, create gene lists and save as 'genes'
for signature in dict_top_genes:
    for direction in ('top', 'bottom'):
        genes = dict_top_genes[signature][direction] # value = dict [1st key] [2nd key]
        results[signature][direction] = enrichr_result_to_df(genes,'','GO_Biological_Process_2018')
        #for signature in list(dict_top_genes.keys())[0]:

# freeze default dict for readonly1
results.default_factory = None

In [201]:
# Number of studies to be processed
n = len(study_names)
i = 0
for study in study_names[:]:
    
df_1_top=pd.DataFrame(results['1']['top'])

In [209]:
# if creating a list instead of dictionary
test = []
for signature in dict_top_genes:
    for direction in ('top', 'bottom'):
        genes = dict_top_genes[signature][direction] # value = dict [1st key] [2nd key]
        genes_df = pd.DataFrame(enrichr_result_to_df(genes,'','GO_Biological_Process_2018'))
        #specify names of each geneset (which study and top/bottom they are from)
        test.append([signature, direction, genes_df])

In [ ]:
for study, d1 in D.items():
    for top, v in d1.items():
wanted_keys = dict_df.columns.get_values().tolist() # The keys you want
dict((k, results[k]) for k in wanted_keys if k in results)    